In [213]:
import time
import boto3
import pandas as pd
import numpy as np
import io
import pymysql
from sqlalchemy import create_engine
from io import BytesIO as StringIO
import os

import findspark                                      #Import library to Search for Spark Installation  

findspark.init()    

from pyspark.sql import SparkSession                  # Import of Spark Session
from pyspark import SparkContext as spark             # Import the Regular Spark Contex
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

from datetime import datetime
from datetime import timedelta

import pyarrow.parquet as pq                          #Create Parquet File
import pyarrow as pa

sc = spark.sparkContext    

import gc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

------

# Creating Variables

## Database

In [214]:
ATHENA_SPG = os.environ['SPG_DATABASE']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [215]:
ATHENA_AUDIT = os.environ['SPG_AUDIT_DATABASE']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Buckets

In [216]:
SPG_INTEGRATION_INPUT_BUCKET = os.environ['INTEGRATION_INPUT_BUCKET']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [217]:
SPG_QUERY_BUCKET = os.environ['QUERY_BUCKET']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [218]:
SPG_OUTPUT_BUCKET = os.environ['OUTPUT_BUCKET']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [219]:
SPG_INPUT_BUCKET = os.environ['INPUT_BUCKET_FROM_OUTPUT']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Input Paths

In [220]:
SPG_OUTPUT_BUCKET_RETENTION = "SPG_AUDIT/SPG_AD_RETETION/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [221]:
SPG_OUTPUT_BUCKET_PURGE = "SPG_AUDIT/SPG_AD_PURGE/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [222]:
SPG_QUERY_BUCKET_ATHENA = "SPG_QUERY/SPG_ATHENA"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [223]:
QUERY_AUDIT = "SELECT ID, timestamp as 'Data Coleta', "\
                "filial as 'Filial',"\
                "cod_vendedor as 'Codigo Vendedor',"\
                "nome_vendedor as 'Vendedor', "\
                "cod_cliente as 'Codigo Cliente',"\
                "nome_cliente as 'Cliente',"\
                "cod_product as 'SKU',"\
                "desc_product as 'Produto',"\
                "ROUND(COALESCE(preco21diaskg,0),2) as 'Preco',"\
                "fabricante as 'Fabricante',"\
                "cidade as 'Cidade',"\
                "estado as 'Estado',"\
                "ovajustada as 'OV',"\
                "aprovado_manualmente as 'Aprovado Manualmente', "\
                "ROUND(COALESCE(preco_ref_min,0),2) as 'Preco Ref. Minimo',"\
                "ROUND(COALESCE(preco_ref_max,0),2) as 'Preco Ref. Max' "\
                "from "+ATHENA_AUDIT+".vw_auditoria_preco_referencia "\
                "where (aprovado_automaticamente = 'Não' or aprovado_automaticamente is null) and "\
                "aprovado_manualmente is null and "\
                "timestamp >= '{}'"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [224]:
QUERY_COLLECT = "SELECT "\
                "TIMESTAMP,"\
                "COD_SALES_AGENT as cod_vendedor,"\
                "SALES_AGENT as nome_vendedor,"\
                "COD_CLIENT as cod_cliente,"\
                "CLIENT_NAME as nome_cliente,"\
                "COD_PRODUCT as cod_product,"\
                "DESC_PRODUCT as desc_product,"\
                "GPM as gpm,"\
                "COMPETITOR as fabricante,"\
                "PRICE_WITH_TAXES as preco_com_impostos,"\
                "PAYMENT_TERM as prazo_de_pagamento,"\
                "PRICE_TYPE as tipo_de_proce,"\
                "UNIT_MEASURE as unidade_de_medida,"\
                "CITY as cidade,"\
                "STATE as estado,"\
                "ADJ_SALES_ORG as ovajustada,"\
                "MATERIAL as material,"\
                "PRICE_21_DAYS_KG as preco21diaskg,"\
                "MONTH_YEAR as mesano,"\
                "PASS_DAYS as diascorridos,"\
                "REMAINING_DAYS as diasfaltantes,"\
                "KEY_SELLERS_MARKS as chavevendedormetas,"\
                "APPROVED as aprovado,"\
                "BRANCH as filial,"\
                "REGION as regiao,"\
                "PRACTICED_PRICE_KG_MEDIAN_AVG as median_avg_precopraticadokg,"\
                "COUNT_BILLING_QTT as count_qtd_fatura,"\
                "WEEK_PRICE as precosemana,"\
                "DESC_GPD as desc_gpd,"\
                "CURRENT_STATUS as statusatual,"\
                "ADJ_PRICE_TYPE as tipo_de_preco_ajustes,"\
                "ANALYSIS_UNIT as unidade_de_analise,"\
                "COMPETITOR_PRICE_STD_DEVIATION as stddev_precoconcorrente"\
            "FROM tb_coleta "\
            "where cast(from_iso8601_timestamp(timestamp) as date) >= cast('{}' as date)"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [225]:
QUERY_GET_TIMESTAMP = "SELECT max(timestamp) as data_ref FROM tb_coleta"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Output Paths

In [226]:
# unknown

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Boto3 Variable

In [227]:
#S3 Configuration
S3_ATHENA_INPUT =  's3://'+SPG_QUERY_BUCKET+'/'+SPG_QUERY_BUCKET_ATHENA

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [228]:
S3_ATHENA_OUTPUT = 's3://'+SPG_QUERY_BUCKET+'/'+SPG_QUERY_BUCKET_ATHENA

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [229]:
region_name = os.environ['AWS_REGION']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [230]:
aws_access_key_id = os.environ['AWS_ACCESS_KEY']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [231]:
aws_secret_access_key = os.environ['AWS_SECRET_KEY']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Preparing Classes

In [232]:
class QueryAthena:

    def __init__(self, query, database):
        self.database = database
        self.folder = SPG_QUERY_BUCKET_ATHENA+"/" 
        self.bucket = SPG_OUTPUT_BUCKET
        self.s3_input = 's3://' + self.bucket + '/my_folder_input'
        self.s3_output =  's3://' + self.bucket + '/' + self.folder
        self.region_name = region_name
        self.aws_access_key_id = aws_access_key_id
        self.aws_secret_access_key = aws_secret_access_key
        self.query = query

    def load_conf(self, q):
        try:
            self.client = boto3.client('athena', 
                              region_name = self.region_name, 
                              aws_access_key_id = self.aws_access_key_id,
                              aws_secret_access_key= self.aws_secret_access_key)
            response = self.client.start_query_execution(
                QueryString = q,
                    QueryExecutionContext={
                    'Database': self.database
                    },
                    ResultConfiguration={
                    'OutputLocation': self.s3_output,
                    }
            )
            self.filename = response['QueryExecutionId']
            return response
            #print('Execution ID: ' + response['QueryExecutionId'])

        except Exception as e:
            log_table = spark.createDataFrame([(1, 2)], ("a", "b"))              #Spark Dataframe with test columns

    def run_query(self):
        queries = [self.query]
        for q in queries:
            res = self.load_conf(q)
            
        df = None
        try:              
            query_status = None
            while query_status == 'QUEUED' or query_status == 'RUNNING' or query_status is None:
                query_status = self.client.get_query_execution(QueryExecutionId=res["QueryExecutionId"])['QueryExecution']['Status']['State']
                if query_status == 'FAILED' or query_status == 'CANCELLED':

                    e = 'Athena query with the string "{}" failed or was cancelled'.format(self.query)
                time.sleep(10)
            #print('Query "{}" finished.'.format(self.query))
            df = self.obtain_data()
            self.remove_data()
            return df

        except Exception as e:
            self.remove_data()
            log_table = spark.createDataFrame([(1, 2)], ("a", "b"))              #Spark Dataframe with test columns

    def obtain_data(self):
        try:
            self.resource = boto3.resource('s3', 
                                  region_name = self.region_name, 
                                  aws_access_key_id = self.aws_access_key_id,
                                  aws_secret_access_key= self.aws_secret_access_key)

            response = self.resource \
            .Bucket(self.bucket) \
            .Object(key= self.folder + self.filename + '.csv') \
            .get()

            return pd.read_csv(io.BytesIO(response['Body'].read()), encoding='utf8')   
        except Exception as e:
            log_table = spark.createDataFrame([(1, 2)], ("a", "b"))              #Spark Dataframe with test columns
     
    def remove_data(self):
        try:
            self.resource = boto3.resource('s3', 
                                  region_name = self.region_name, 
                                  aws_access_key_id = self.aws_access_key_id,
                                  aws_secret_access_key= self.aws_secret_access_key)            
            self.resource.Object(self.bucket, self.folder + self.filename + '.csv').delete()
            self.resource.Object(self.bucket, self.folder + self.filename + '.csv.metadata').delete()
            
        
        except Exception as e:
            log_table = spark.createDataFrame([(1, 2)], ("a", "b"))              #Spark Dataframe with test columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [233]:
class Transfer:
    def __init__(self) :
        self.target_database = ATHENA_AUDIT
        self.target_host = os.environ['AUDIT_HOST']
        self.target_user = os.environ["AUDIT_USER"]
        self.target_passwd = os.environ["AUDIT_PWD"]
        self.target_table = "auditoria"
        self.if_exists = "append"
        
    def doIt(self, dataframe):        
        
        try:
            engine = create_engine('mysql+pymysql://{}:{}@{}/{}'.format(self.target_user,self.target_passwd,self.target_host,self.target_database))
            dataframe.to_sql(con=engine, name=self.target_table, if_exists=self.if_exists, index=False)
        except Exception as e:
            log_table = spark.createDataFrame([(1, 2)], ("a", "b"))              #Spark Dataframe with test columns
            
    def runCommandOnTarget(self, query):
              
        try:
        
            engine = create_engine('mysql+pymysql://{}:{}@{}/{}'.format(self.target_user,self.target_passwd,self.target_host,self.target_database))
            con = engine.connect()
            result = con.execute(query)
            
        except Exception as e:
            log_table = spark.createDataFrame([(1, 2)], ("a", "b"))              #Spark Dataframe with test columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [234]:
class Purge:
    def __init__(self):
        self.database = ATHENA_AUDIT
        self.host = os.environ['AUDIT_HOST']
        self.user = os.environ["AUDIT_USER"]
        self.passwd = os.environ["AUDIT_PWD"]
        self.folder = SPG_OUTPUT_BUCKET_PURGE
        self.bucket = SPG_OUTPUT_BUCKET        
        self.s3_input = 's3://' + self.bucket + '//' + self.folder
        self.s3_output =  's3://' + self.bucket + '//' + self.folder


        self.region_name = region_name
        self.aws_access_key_id = aws_access_key_id
        self.aws_secret_access_key = aws_secret_access_key                
        
    def createPugedFile(self, query, filename):
        try:                    
            #dataframe
            engine = create_engine('mysql+pymysql://{}:{}@{}/{}'.format(self.user,self.passwd,self.host,self.database))
            purger_df = pd.read_sql_query(query, engine)                                
                                    
            if purger_df.shape[0] > 0:
                #buffer
                csv_buffer = StringIO()
                #writing to buffer
                purger_df.to_csv(csv_buffer, sep="|", index=False, encoding='cp1252')                                                      
                
                #connect to s3
                s3 = boto3.resource('s3')
                # Write buffer to S3 
                s3.Object(self.bucket, self.folder+filename).put(Body=csv_buffer.getvalue())                   
                            
        except Exception as e:
            log_table = spark.createDataFrame([(1, 2)], ("a", "b"))              #Spark Dataframe with test columns
            
    def createPugedFileWithExcel(self, query, filename):
        try:
            
            #dataframe
            engine = create_engine('mysql+pymysql://{}:{}@{}/{}'.format(self.user,self.passwd,self.host,self.database))
            purger_df = pd.read_sql_query(query, engine)   
            
            if purger_df.shape[0] > 0:
                output = io.BytesIO()
                writer = pd.ExcelWriter(output, engine='openpyxl')
              #  writer = pd.ExcelWriter(output, engine='xlsxwriter')
                
                purger_df.to_excel(writer,encoding='cp1252',index=False)
                
                writer.save()
                data = output.getvalue()

                s3 = boto3.resource('s3')
                s3.Bucket(self.bucket).put_object(Key=self.folder+filename, Body=data)
        except Exception as e:
            log_table = spark.createDataFrame([(1, 2)], ("a", "b"))              #Spark Dataframe with test columns

    def loadAdjustedPugedRecords(self, filename):
        try:
            s3_file = filename
            s3 = boto3.client('s3')
            
            obj = s3.get_object(Bucket=self.bucket, Key=self.folder+s3_file)
            
            purget_df = pd.read_csv(io.BytesIO(obj['Body'].read()),sep='|',  encoding = "cp1252")
            
            return purget_df                           
            #for item in purget_df.rows:                            
        except Exception as e:
            log_table = spark.createDataFrame([(1, 2)], ("a", "b"))              #Spark Dataframe with test columns

    def loadAdjustedPugedRecordsWithExcel(self, filename):

        try:
            s3_file = filename
            s3 = boto3.client('s3')
            
            obj = s3.get_object(Bucket=self.bucket, Key=self.folder+s3_file)
            
            purget_df = pd.read_excel(io.BytesIO(obj['Body'].read()))
            
            return purget_df                           
            #for item in purget_df.rows:                            
        except Exception as e:
            log_table = spark.createDataFrame([(1, 2)], ("a", "b"))              #Spark Dataframe with test columns
    
    def write_pandas_parquet_to_s3(self, df, fileName):
        try:
        
            #buffer
            csv_buffer = StringIO()
            #writing to buffer
            df.to_parquet(csv_buffer)                                                      
                
            #connect to s3
            s3 = boto3.resource('s3')
            # Write buffer to S3 
            s3.Object(self.bucket, folder_ret+filename).put(Body=csv_buffer.getvalue())
            
        except Exception as e:
            log_table = spark.createDataFrame([(1, 2)], ("a", "b"))              #Spark Dataframe with test columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [235]:
qa = QueryAthena(query=QUERY_GET_TIMESTAMP, database=ATHENA_SPG)
df2 = qa.run_query()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [236]:
data_ref = df2['data_ref'].values[0]

if data_ref == datetime.now().date().strftime('%Y-%m-%d'):
    data_ref = (datetime.strptime(data_ref, '%Y-%m-%d') - timedelta(days=1)).date().strftime('%Y-%m-%d')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [237]:
def cleanup(session, params):
    s3 = session.resource('s3')
    my_bucket = s3.Bucket(params['bucket'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [238]:
date_prior = datetime.strptime(data_ref, '%Y-%m-%d') - timedelta(days=7)
#day = datetime.strptime(data_ref, '%Y-%m-%d')
day = datetime.strptime(data_ref, '%Y-%m-%d') - timedelta(days=1)


filename = "tbl_SPG_AD_PURGE_{}.xlsx".format(day.strftime('%Y_%m_%d'))

sql = u"update "+ATHENA_AUDIT+".auditoria set aprovado_manualmente = '{apr_man}' "\
      "where ID={id_} and `timestamp` >= '{date_prior_}'"


purged_audit = Purge()
purget_df = purged_audit.loadAdjustedPugedRecordsWithExcel(filename)

if purget_df is not None:

    purget_df.rename(columns = {"Data Coleta":"timestamp",
                                "Filial":"filial",
                                "Vendedor":"nome_vendedor",
                                "Cliente":"nome_cliente",
                                "SKU":"cod_product",
                                "Produto":"desc_product",
                                "Preco":"preco21diaskg",
                                "Fabricante":"fabricante",
                                "Cidade":"cidade",
                                "Estado":"estado",
                                "Material":"material",
                                "OV":"ovajustada",
                                "Aprovado Manualmente":"aprovado_manualmente",
                                "Preco Ref. Minimo": "preco_ref_min",
                                "Preco Ref. Max":"preco_ref_max"
                               }, inplace=True)
    ta = Transfer()
    for index,row in purget_df.iterrows():
        ta.runCommandOnTarget(sql.format(apr_man=row['aprovado_manualmente'],id_=row['ID'],date_prior_=date_prior.date().strftime('%Y-%m-%d')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [239]:
qa = QueryAthena(query=QUERY_COLLECT.format(data_ref), database=ATHENA_SPG)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [240]:
coleta = qa.run_query()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [243]:
coleta_tratada1 =  coleta[(~coleta.cod_vendedor.isna()) & (~coleta.cod_cliente.isna()) &
                  (~coleta.filial.isna()) & (~coleta.preco21diaskg.isna())]

coleta_tratada = coleta_tratada1[(coleta_tratada1.cod_vendedor.notnull()) & 
                                 (coleta_tratada1.cod_cliente.notnull()) &
                                 (coleta_tratada1.filial.notnull()) & 
                                 (coleta_tratada1.preco21diaskg.notnull())]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'NoneType' object has no attribute 'cod_vendedor'
Traceback (most recent call last):
AttributeError: 'NoneType' object has no attribute 'cod_vendedor'



In [208]:
coleta_tratada.drop(columns = ["aprovado"], inplace=True)
coleta_tratada['aprovado_manualmente'] = None
coleta_tratada['aprovado_automaticamente'] = 'Não'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'coleta_tratada' is not defined
Traceback (most recent call last):
NameError: name 'coleta_tratada' is not defined



In [209]:
ta = Transfer()

sql = "delete from "+ATHENA_AUDIT+".auditoria where cast(timestamp as date) = cast('{}' as date)"

sql2 = sql.format(data_ref)

clear_target= ta.runCommandOnTarget(sql2)

ta.doIt(coleta_tratada)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

name 'coleta_tratada' is not defined
Traceback (most recent call last):
NameError: name 'coleta_tratada' is not defined



In [49]:
retencao = coleta[(coleta.cod_vendedor.isna()) | (coleta.cod_cliente.isna()) |
                  (coleta.filial.isna()) | (coleta.preco21diaskg.isna()) |
                  (coleta.cod_vendedor.isnull()) | (coleta.cod_cliente.isnull()) |
                  (coleta.filial.isnull()) | (coleta.preco21diaskg.isnull())
                 ]
 
filename = "tbl_SPG_AD_RET_{}.parquet".format(data_ref)

purged_audit = Purge()
purged_audit.write_pandas_parquet_to_s3(retencao, filename)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
date_prior = datetime.strptime(data_ref, '%Y-%m-%d') - timedelta(days=7)
day = datetime.strptime(data_ref, '%Y-%m-%d')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
#Atualizando status da aprovação de registros acima de 7 dias no mysql
ta = Transfer()

sql = "update "+ATHENA_AUDIT+".auditoria set aprovado_manualmente='Não' where "\
      "timestamp < {} and aprovado_manualmente is null"

upd= ta.runCommandOnTarget(sql.format(date_prior.date().strftime('%Y-%m-%d')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
#filename = "tbl_SPG_AD_PURGE_{}.csv".format(day.strftime('%Y_%m_%d'))
filename = "tbl_SPG_AD_PURGE_{}.xlsx".format(day.strftime('%Y_%m_%d'))

query = QUERY_AUDIT

#print(query.format(date_prior.date().strftime('%Y-%m-%d')))
purged_audit = Purge()
purged_audit.createPugedFileWithExcel(query.format(date_prior.date().strftime('%Y-%m-%d')), filename)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…